# **CrossPTM V1.0**
Please set 'Runtime->Change runtime type' to GPU and hit Runtime -> Run all



**Step 1: Download code and checkpoint from Github.**


In [2]:
# !git clone https://github.com/hanye311/CrossPTM.git
!git clone https://github.com/Yichuan0712/CrossPTM.git
%cd CrossPTM
!pip -q install gdown

FILE_ID = "1ldPrBSvkjqprzIlo4EkA_nALHieCiVEN"
OUT = "/content/CrossPTM/best_model_yichuan.pth"

!gdown --fuzzy "https://drive.google.com/uc?id={FILE_ID}" -O "{OUT}"

import os
print("size:", os.path.getsize(OUT), "bytes")

Cloning into 'CrossPTM'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 66 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 828.03 KiB | 16.24 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/CrossPTM
Downloading...
From (original): https://drive.google.com/uc?id=1ldPrBSvkjqprzIlo4EkA_nALHieCiVEN
From (redirected): https://drive.google.com/uc?id=1ldPrBSvkjqprzIlo4EkA_nALHieCiVEN&confirm=t&uuid=a5df3d62-0972-48b1-a771-6b489f435451
To: /content/CrossPTM/best_model_yichuan.pth
100% 3.26G/3.26G [00:09<00:00, 347MB/s]
size: 3258639848 bytes



**Step 2: Install necessary package.**


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install fair-esm biopython -q
!pip install pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.7 MB/s eta 0:00:00


In [6]:
!cp /content/drive/MyDrive/env/ngrok_authtoken.txt /content/ngrok_authtoken.txt

In [7]:
from flask import Flask, render_template, jsonify, request, send_file
from pyngrok import ngrok
app = Flask(__name__)

PORT = 5000

NGROK_TOKEN_PATH = "/content/ngrok_authtoken.txt"
if os.path.exists(NGROK_TOKEN_PATH):
    with open(NGROK_TOKEN_PATH, "r") as f:
        token = f.read().strip()
    if token:
        ngrok.set_auth_token(token)
else:
    print(f"[WARN] ngrok token file not found at: {NGROK_TOKEN_PATH}")

**Step 3: Run prediction tool(Please set 'Runtime->Change runtime type' to GPU)**

1.   Input or upload a FASTA file。
2.   Choose the PTM type you want to predict.
3.   Click the "Start Prediction" button.
4.   Wait for the results (⚠️ Please do not click "Start Prediction" more than once).
5.   Download the results.


In [ ]:
import os
import subprocess
import tempfile
import uuid
from io import BytesIO

import pandas as pd
import requests
from flask import Flask, render_template_string, request, send_file, url_for
from pyngrok import ngrok

# ================== Basic config ==================
PORT = 5000

NGROK_TOKEN_PATH = "/content/ngrok_authtoken.txt"
if os.path.exists(NGROK_TOKEN_PATH):
    with open(NGROK_TOKEN_PATH, "r") as f:
        token = f.read().strip()
    if token:
        ngrok.set_auth_token(token)
else:
    print(f"[WARN] ngrok token file not found at: {NGROK_TOKEN_PATH}")

# PTM options (Sumoylation_K first)
PTM_OPTIONS = [
    'Sumoylation_K',
    'Phosphorylation_ST', 'Phosphorylation_Y',
    'Methylation_K', 'Acetylation_K', 'Ubiquitination_K',
    'NlinkedGlycosylation_N',
    'Methylation_R'
]
DEFAULT_PTM = 'Sumoylation_K'

# Example FASTA sequence (same as in your PTMPredictor.load_example)
EXAMPLE_FASTA = """>4EBP_DROME
MSASPTARQAITQALPMITRKVVISDPIQMPEVYSSTPGGTLYSTTPGGTKLIYERAFMKNLRGSPLSQTPPSNVPSCLLRGTPRTPFRKCVPVPTELIKQTKSLKIEDQEQFQLDL"""

# Mapping from PTM type to GitHub raw image URL
PTM_IMAGE_MAP = {
    'Acetylation_K': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Acetylation_K.png',
    'Methylation_K': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Methylation_K.png',
    'Methylation_R': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Methylation_R.png',
    'NlinkedGlycosylation_N': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/NlinkedGlycosylation_N.png',
    'Phosphorylation_ST': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Phosphorylation_ST.png',
    'Phosphorylation_Y': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Phosphorylation_Y.png',
    'Sumoylation_K': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Sumoylation_K.png',
    'Ubiquitination_K': 'https://github.com/hanye311/CrossPTM/raw/main/sequence_logo/Ubiquitination_K.png',
}

app = Flask(__name__)
# Store token -> csv file path
RESULTS_STORE = {}


# ================== Helper functions ==================
def validate_fasta(fasta_content: str):
    """Validate FASTA format."""
    if not fasta_content.strip():
        return False, "Please enter FASTA sequences."

    lines = fasta_content.strip().split('\n')
    if not any(line.startswith('>') for line in lines):
        return False, "FASTA format error: missing sequence identifiers (lines starting with '>')."

    return True, "Format is correct."


def save_temp_fasta(fasta_content: str):
    """Save a temporary FASTA file and return its path."""
    temp_dir = tempfile.mkdtemp()
    fasta_path = os.path.join(temp_dir, 'input_sequences.fasta')
    with open(fasta_path, 'w') as f:
        f.write(fasta_content)
    return fasta_path


def run_model(fasta_content: str, ptm_type: str):
    """
    Run test.py model.
    Returns:
        (True, df, csv_path) on success
        (False, error_message_str, None) on failure
    """
    fasta_path = save_temp_fasta(fasta_content)
    output_dir = tempfile.mkdtemp()

    cmd = [
        'python', 'test.py',
        '--config_filename', 'config/test.yaml',
        '--model_path', 'best_model_yichuan.pth',
        '--data_path', fasta_path,
        '--PTM_type', ptm_type,
        '--save_path', output_dir
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
    except Exception as e:
        return False, f"Error when running model: {e}", None

    if result.returncode != 0:
        msg = "Prediction failed (return code: {})<br><pre>{}</pre>".format(
            result.returncode, (result.stderr or result.stdout)
        )
        return False, msg, None

    # Find CSV
    csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]
    if not csv_files:
        msg = "No output CSV file found.<br><pre>{}</pre>".format(result.stdout)
        return False, msg, None

    csv_path = os.path.join(output_dir, csv_files[0])

    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        msg = f"Error reading CSV: {e}"
        return False, msg, None

    return True, df, csv_path


# ================== HTML template ==================
INDEX_HTML = r"""
<!doctype html>
<html>
<head>
    <meta charset="utf-8">
    <title>PTM Prediction Tool</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 960px;
            margin: 20px auto;
            line-height: 1.6;
        }
        h1, h2, h3 {
            color: #333;
        }
        textarea {
            width: 100%;
            font-family: monospace;
        }
        .btn {
            padding: 8px 16px;
            border-radius: 4px;
            border: none;
            cursor: pointer;
            margin-right: 10px;
            min-width: 180px;
        }
        .btn-primary {
            background-color: #007bff;
            color: white;
        }
        .btn-primary:hover {
            background-color: #0056b3;
        }
        .status {
            margin-top: 20px;
            padding: 10px;
            border-radius: 4px;
        }
        .status-ok {
            background-color: #e6ffed;
            border: 1px solid #2ecc71;
        }
        .status-bad {
            background-color: #ffecec;
            border: 1px solid #e74c3c;
        }
        table {
            border-collapse: collapse;
            width: 100%;
            margin-top: 10px;
            font-size: 13px;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 4px 6px;
        }
        th {
            background-color: #f7f7f7;
        }
        .download-link {
            margin-top: 10px;
        }
        .file-name {
            font-size: 0.9rem;
            color: #555;
            margin-left: 8px;
        }
    </style>
    <script>
        function loadExample() {
            const example = {{ example_fasta | tojson }};
            const textarea = document.getElementById('fasta_text');
            textarea.value = example;
        }
        function triggerFileInput() {
            document.getElementById('fasta_file').click();
        }
        function updateFileName(input) {
            const label = document.getElementById('file_name');
            if (input.files && input.files.length > 0) {
                label.textContent = input.files[0].name;
            } else {
                label.textContent = "";
            }
        }
    </script>
</head>
<body>
    <h1>PTM Prediction Tool</h1>
    <hr>

    <form method="post" enctype="multipart/form-data">
        <h3>FASTA Sequence Input (text)</h3>
        <textarea id="fasta_text" name="fasta_text" rows="10" placeholder="&gt;example
MSEQUENCE">{{ (request.form.get('fasta_text') or '') }}</textarea>
        <p style="margin-top: 8px;">
            <button type="button" class="btn btn-primary" onclick="loadExample()">Load Example</button>
        </p>

        <h3>Upload FASTA File</h3>
        <input type="file"
               id="fasta_file"
               name="fasta_file"
               accept=".fasta,.fa,.txt"
               style="display: none;"
               onchange="updateFileName(this)">
        <button type="button" class="btn btn-primary" onclick="triggerFileInput()">Choose FASTA File</button>
        <span id="file_name" class="file-name"></span>

        <h3>Parameters</h3>
        <label for="ptm_type">PTM Type:</label>
        <select name="ptm_type" id="ptm_type">
            {% for opt in ptm_options %}
            <option value="{{ opt }}"
                {% if opt == (request.form.get('ptm_type') or default_ptm) %}selected{% endif %}>
                {{ opt }}
            </option>
            {% endfor %}
        </select>

        <p style="margin-top:15px;">
            <button type="submit" class="btn btn-primary">Start Prediction</button>
        </p>
    </form>

    {% if message %}
        <div class="status {% if success %}status-ok{% else %}status-bad{% endif %}">
            {{ message|safe }}
        </div>
    {% endif %}

    {% if table_html %}
        <h3>Prediction Results Preview (first 20 rows)</h3>
        <div>{{ table_html|safe }}</div>
    {% endif %}

    {% if image_url %}
        <h3>PTM-specific sequence illustration</h3>
        <img src="{{ image_url }}" alt="PTM image"
             style="max-width: 100%; height: auto; border: 1px solid #ccc; margin-top: 10px;">
    {% endif %}

    {% if download_url %}
        <div class="download-link">
            <form method="get" action="{{ download_url }}">
                <button type="submit" class="btn btn-primary">Download Full CSV Results</button>
            </form>
        </div>
    {% endif %}
</body>
</html>
"""


# ================== Routes ==================
@app.route("/", methods=["GET", "POST"])
def index():
    message = None
    table_html = None
    download_url = None
    image_url = None
    success_flag = False

    if request.method == "POST":
        ptm_type = request.form.get("ptm_type", DEFAULT_PTM)

        # Prefer uploaded file; if not present, use textarea content
        uploaded = request.files.get("fasta_file")
        fasta_text = (request.form.get("fasta_text") or "").strip()

        if uploaded and uploaded.filename:
            try:
                fasta_content = uploaded.read().decode("utf-8")
            except Exception as e:
                message = f"Failed to read uploaded file: {e}"
                return render_template_string(
                    INDEX_HTML,
                    message=message,
                    table_html=None,
                    download_url=None,
                    image_url=None,
                    ptm_options=PTM_OPTIONS,
                    default_ptm=DEFAULT_PTM,
                    success=False,
                    example_fasta=EXAMPLE_FASTA,
                )
        else:
            fasta_content = fasta_text

        # Validate FASTA
        is_valid, msg = validate_fasta(fasta_content)
        if not is_valid:
            message = msg
            return render_template_string(
                INDEX_HTML,
                message=message,
                table_html=None,
                download_url=None,
                image_url=None,
                ptm_options=PTM_OPTIONS,
                default_ptm=DEFAULT_PTM,
                success=False,
                example_fasta=EXAMPLE_FASTA,
            )

        # Run model
        ok, res, csv_path = run_model(fasta_content, ptm_type)
        if not ok:
            # res is an error message string
            message = res
            success_flag = False
        else:
            df = res
            success_flag = True
            message = f"Prediction completed. Total records: {len(df)}"
            table_html = df.head(20).to_html(index=False, escape=False)

            # Create download token
            token = str(uuid.uuid4())
            RESULTS_STORE[token] = csv_path
            download_url = f"/download/{token}"

            # Image URL for selected PTM type (served by our own route)
            if ptm_type in PTM_IMAGE_MAP:
                image_url = url_for('ptm_image', ptm_type=ptm_type)

    return render_template_string(
        INDEX_HTML,
        message=message,
        table_html=table_html,
        download_url=download_url,
        image_url=image_url,
        ptm_options=PTM_OPTIONS,
        default_ptm=DEFAULT_PTM,
        success=success_flag,
        example_fasta=EXAMPLE_FASTA,
    )


@app.route("/download/<token>")
def download(token):
    """Download CSV results by token."""
    csv_path = RESULTS_STORE.get(token)
    if not csv_path or not os.path.exists(csv_path):
        return "File not found", 404

    return send_file(csv_path, as_attachment=True, download_name=os.path.basename(csv_path))


@app.route("/ptm_image/<ptm_type>")
def ptm_image(ptm_type):
    """Fetch PTM-specific image from GitHub and serve it through this app."""
    url = PTM_IMAGE_MAP.get(ptm_type)
    if not url:
        return "Image not found", 404

    try:
        resp = requests.get(url, timeout=10)
    except Exception as e:
        return f"Failed to fetch image: {e}", 500

    if resp.status_code != 200:
        return "Image not found", 404

    img_bytes = BytesIO(resp.content)
    img_bytes.seek(0)
    return send_file(img_bytes, mimetype="image/png")


@app.route("/health")
def health():
    return "OK", 200


# ================== Start Flask + ngrok ==================
# Kill previous tunnels to avoid duplicates
try:
    ngrok.kill()
except Exception:
    pass

public_url = ngrok.connect(PORT).public_url
print(" * ngrok public URL:", public_url)
print(" * Open this URL in your browser to access the PTM Prediction web interface.")

# Disable reloader for notebook usage
app.run(port=PORT, debug=False, use_reloader=False)


 * ngrok public URL: https://886d6002de3b.ngrok-free.app
 * Open this URL in your browser to access the PTM Prediction web interface.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 22:23:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 22:23:55] "GET /favicon.ico HTTP/1.1" 404 -
